In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import models
 
# # Create the model
model = models.Sequential()
# # model.add(layers.Conv2D(64, (11,11), padding='valid',activation='relu',   input_shape=(128, 128, 3))) # νομιζω ειναι το καλυτερο μου
# # model.add(layers.Conv2D(64, (11,11),  padding='valid', activation='relu'))
# # model.add(layers.Conv2D(64, (11,11),padding='valid',   activation='relu'))
# # model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))
          
# # model.add(layers.Conv2D(128, (11,11), padding='valid', activation='relu')) 
# # model.add(layers.Conv2D(128, (11,11), padding='valid',  activation='relu')) 
# # model.add(layers.Conv2D(128, (11,11), padding='valid',  activation='relu')) 
# # model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))


model.add(tf.keras.layers.Conv2D(64,  (11,11), padding='same', activation='relu', input_shape=(128, 128, 3)))
#model.add(layers.Conv2D(64,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

# model.add(layers.Conv2D(128,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(128,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(128,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))
# model.add(layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))
# model.add(layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))




         
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      23296     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       991360    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 256)       3

In [2]:
import os
base_dir = os.getcwd()
base_dir = os.path.join(base_dir, r'duth-cv-2019-2020-hw-4\vehicles')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255 , rotation_range = 10 , width_shift_range = 0.1 , height_shift_range = 0.1 , shear_range = 0.15 , zoom_range = 0.1 , channel_shift_range = 10. , horizontal_flip = True)
val_datagen  = ImageDataGenerator(rescale=1./255, rotation_range = 10 , width_shift_range = 0.1 , height_shift_range = 0.1 , shear_range = 0.15 , zoom_range = 0.1 , channel_shift_range = 10. , horizontal_flip = True)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=50,
                                                    class_mode='categorical',
                                                    # color_mode='grayscale',
                                                    target_size=(128,128),
                                                    shuffle=True)     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  val_datagen.flow_from_directory(validation_dir,
                                                        batch_size=50,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(128,128)) 

Found 2494 images belonging to 6 classes.
Found 311 images belonging to 6 classes.


In [3]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

NameError: name 'Model' is not defined

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image

import os
import csv

model = tf.keras.models.load_model('small_last4.h5')
rowlist = [['Id', 'Category']]

for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames:
        
        path = os.path.join(dirname, filename)
        img = image.load_img(path, target_size=(128, 128), grayscale=False, interpolation='bilinear')
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        classes_pred = model.predict(x)
        cls_pred = np.argmax(classes_pred)
        rowlist.append([filename, cls_pred])
        #print(filename, cls_pred)
        with open('output.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rowlist)
        

1002879756_98b62b8b77.jpg
100574956_ccfcc18d73.jpg
1013926955_d33ef1fb75.jpg
101785441_67ab962d33.jpg
102167461_92db247353.jpg
103548670_2c7ca7e928.jpg
1037210243_318922a9f6.jpg
1049852114_28795f1503.jpg
1063663572_d2071882a7.jpg
10649366_ef9dbfd34a.jpg
1071025208_acf022059d.jpg
107759625_d86d8b8a0b.jpg
1081512254_879475ed60.jpg
1084589570_b3dd42f693.jpg
108503753_971c399b31.jpg
1097887041_8fe45e840a.jpg
1099941359_7ae0572599.jpg
110020961_797ad9cf30.jpg
1113545850_0c906e3058.jpg
1114643352_27ad1fd5cd.jpg
1124733995_cde9a34d46.jpg
112594641_7f1b9a4c5a.jpg
1128932598_a7dd570b88.jpg
1137646735_2fb2752249.jpg
113837123_2af66d651e.jpg
113893980_5518314749.jpg
114324557_207c56f408.jpg
1149157136_c72f22d78c.jpg
11543654_755368c930.jpg
1155266167_46d04fca90.jpg
116462660_81bf7422ee.jpg
1165245408_f6d2a19d98.jpg
117021856_b69d0c622d.jpg
1171035303_8ae3db78a6.jpg
1171835222_0eec3ad516.jpg
1177644899_de9d0f0105.jpg
1185122089_2d4905698b.jpg
118841012_7041556c99.jpg
119027035_e435c5a4af.jpg
11920

# 